### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

---

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

### <font color=red>\*\*\* _Note About Overal Passing Rate_ \*\*\*</font>

The formula that is listed as the overall passing rate is the average of the sum of the math and reading score averages.  This is a score, not a percent or rate of passing, it does not give any information regarding number of student who had scores over the 70 point threshold.  There are a few different ways we could interpret and report this score differently:

    1)  Average of math and reading passing percentages  
    2)  Percentage of students passing both math and reading  
    3)  Percentage of students with scores of 70 or above for the average between their math and reading scores
    
Since the instructions reference summing the math and reading scores for this value, I am interpreting this as the third statement in the list above:  
`Overall Passing Rate = Count of Students Where { ((Math + Reading) / 2) >= 70 } / Count of Students * 100`

In [27]:
    #Create DataFrame of summary statistics
district_summary = pd.DataFrame({
    'Total Schools': [school_data_complete['school_name'].nunique()],   #Number of unique school names 
    'Total Students': [school_data_complete['student_name'].count()],   #Number of students
    'Total Budget': [school_data_complete['budget'].unique().sum()],   #Sum of unique school budget values  *** Assuming that no two budgets are identical ***
    'Average Math Score': [school_data_complete['math_score'].mean()],   #Average of all student math scores
    'Average Reading Score': [school_data_complete['reading_score'].mean()],   #Average of all student reading scores
    '% Passing Math': [school_data_complete[school_data_complete['math_score'] > 69].shape[0] / school_data_complete.shape[0] * 100],   #Number of rows where math score > 69 / total rows * 100
    '% Passing Reading': [school_data_complete[school_data_complete['reading_score'] > 69].shape[0] / school_data_complete.shape[0] * 100],   #Number of rows where reading score > 69 / total rows * 100
    '% Overall Passing Rate': [school_data_complete[(school_data_complete['math_score'] + school_data_complete['reading_score']) / 2 > 69].shape[0] / school_data_complete.shape[0] * 100]   #Number of rows where mean([math, reading]) score > 69 / total rows * 100
})

    #Format results
district_summary['Total Budget'] = district_summary['Total Budget'].map('${:,.2f}'.format)
district_summary['Average Math Score'] = district_summary['Average Math Score'].map('{:.3f}'.format)
district_summary['Average Reading Score'] = district_summary['Average Reading Score'].map('{:.3f}'.format)
district_summary['% Passing Math'] = district_summary['% Passing Math'].map('{:.3f}%'.format)
district_summary['% Passing Reading'] = district_summary['% Passing Reading'].map('{:.3f}%'.format)
district_summary['% Overall Passing Rate'] = district_summary['% Overall Passing Rate'].map('{:.3f}%'.format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985,81.878,74.981%,85.805%,90.444%


---

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [36]:
    #Create DataFrame for summary statistics by school
    #Each instance of '.agg(lambda x: x.unique())' is returning the single unique value (agreggating them) for each groupby index
    #All data is grouped by ['school_name']
school_summary = pd.DataFrame({
    'School Type': school_data_complete.groupby(['school_name'])['type'].agg(lambda x: x.unique()),   #Type by school
    'Total Students': school_data_complete.groupby(['school_name'])['size'].agg(lambda x: x.unique()),   #Student population by school
    'Total School Budget': school_data_complete.groupby(['school_name'])['budget'].agg(lambda x: x.unique()),   #Budget by school
    'Per Student Budget': school_data_complete.groupby(['school_name'])['budget'].agg(lambda x: x.unique()) / school_data_complete.groupby(['school_name'])['size'].agg(lambda x: x.unique()),   #Returns budget / size
    'Average Math Score': school_data_complete.groupby(['school_name'])['math_score'].mean(),   #Average math score by school
    'Average Reading Score': school_data_complete.groupby(['school_name'])['reading_score'].mean(),   #Average reading score by school
    '% Passing Math': school_data_complete[school_data_complete['math_score'] > 69].groupby(['school_name'])['math_score'].count() / school_data_complete.groupby(['school_name'])['size'].agg(lambda x: x.unique()) * 100,   #Number of math scores > 69 / school student population * 100 
    '% Passing Reading': school_data_complete[school_data_complete['reading_score'] > 69].groupby(['school_name'])['reading_score'].count() / school_data_complete.groupby(['school_name'])['size'].agg(lambda x: x.unique()) * 100,   #Number of reading scores > 69 / school student population * 100
    '% Overall Passing Rate': school_data_complete[(school_data_complete['math_score'] + school_data_complete['reading_score']) / 2 > 69].groupby(['school_name'])['student_name'].count() / school_data_complete.groupby(['school_name'])['size'].agg(lambda x: x.unique()) * 100   #Number of scores where mean([math, reading]) > 69 / total rows * 100
})

    #Create copy to contain fomatted data due to formatting converting values to strings
school_summary_formatted = school_summary.copy()

    #Format results
school_summary_formatted['Total School Budget'] = school_summary_formatted['Total School Budget'].map('${:,.2f}'.format)
school_summary_formatted['Per Student Budget'] = school_summary_formatted['Per Student Budget'].map('${:,.2f}'.format)
school_summary_formatted['Average Math Score'] = school_summary_formatted['Average Math Score'].map('{:.3f}'.format)
school_summary_formatted['Average Reading Score'] = school_summary_formatted['Average Reading Score'].map('{:.3f}'.format)
school_summary_formatted['% Passing Math'] = school_summary_formatted['% Passing Math'].map('{:.3f}%'.format)
school_summary_formatted['% Passing Reading'] = school_summary_formatted['% Passing Reading'].map('{:.3f}%'.format)
school_summary_formatted['% Overall Passing Rate'] = school_summary_formatted['% Overall Passing Rate'].map('{:.3f}%'.format)

    #Remove 'school_name' title from index
school_summary_formatted = school_summary_formatted.rename_axis('')

    #Sort results in descending order by '% Overall Passing Rate', display first 5
school_summary_formatted.sort_values(by=['% Overall Passing Rate'], ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062,83.976,94.133%,97.040%,99.785%
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359,83.726,93.867%,95.855%,99.773%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351,83.817,93.392%,97.139%,99.728%
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682,83.955,93.333%,96.611%,99.611%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274,83.989,93.868%,96.540%,99.606%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [17]:
    #Sort results in ascending order by '% Overall Passing Rate', display first 5
school_summary_formatted.sort_values(by=['% Overall Passing Rate']).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712,81.158,65.988%,80.739%,85.961%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.103,80.746,68.310%,79.299%,86.199%
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.843,80.745,66.367%,80.220%,86.247%
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048,81.034,66.680%,81.933%,86.274%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629,81.183,65.684%,81.316%,86.287%


---

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [13]:
    #Create DataFrame for math scores by grade
    #All data is grouped by school
math_by_grade = pd.DataFrame({
    '9th Grade': school_data_complete[school_data_complete['grade'] == '9th'].groupby(['school_name'])['math_score'].mean(),   #Give the average math scores where 'grade' is '9th'
    '10th Grade': school_data_complete[school_data_complete['grade'] == '10th'].groupby(['school_name'])['math_score'].mean(),   #Give the average math scores where 'grade' is '10th'
    '11th Grade': school_data_complete[school_data_complete['grade'] == '11th'].groupby(['school_name'])['math_score'].mean(),   #Give the average math scores where 'grade' is '11th'
    '12th Grade': school_data_complete[school_data_complete['grade'] == '12th'].groupby(['school_name'])['math_score'].mean()   #Give the average math scores where 'grade' is '12th'
})

    #Format results
    #Since all results are formatted in the same way, one function can be used for everything instead of applying series by series
math_by_grade = math_by_grade.applymap('{:.2f}'.format).rename_axis('')

math_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [14]:
    #Create DataFrame for math scores by grade
    #All data is grouped by school
reading_by_grade = pd.DataFrame({
    '9th Grade': school_data_complete[school_data_complete['grade'] == '9th'].groupby(['school_name'])['reading_score'].mean(),   #Give the average reading scores where 'grade' is '9th'
    '10th Grade': school_data_complete[school_data_complete['grade'] == '10th'].groupby(['school_name'])['reading_score'].mean(),   #Give the average reading scores where 'grade' is '10th'
    '11th Grade': school_data_complete[school_data_complete['grade'] == '11th'].groupby(['school_name'])['reading_score'].mean(),   #Give the average reading scores where 'grade' is '11th'
    '12th Grade': school_data_complete[school_data_complete['grade'] == '12th'].groupby(['school_name'])['reading_score'].mean()   #Give the average reading scores where 'grade' is '12th'
})

    #Format results
    #Since all results are formatted in the same way, one function can be used for everything instead of applying series by series
reading_by_grade = reading_by_grade.applymap('{:.2f}'.format).rename_axis('')

reading_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


---

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [15]:
    #Define bin ranges and labels
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [16]:
    #Create series of what bin range each each row falls under by 'Per Student Budget'
    #Append 'school_summary' DataFrame with series
school_summary['Spending Ranges (Per Student)'] = pd.cut(school_summary['Per Student Budget'],spending_bins,labels=group_names)

    #Create DataFrame of student score summary
    #All data is grouped by per student spending range
Score_by_Spending = pd.DataFrame({
    'Average Math Score': school_summary.groupby(['Spending Ranges (Per Student)'])['Average Math Score'].mean(),   #Average of math scores
    'Average Reading Score': school_summary.groupby(['Spending Ranges (Per Student)'])['Average Reading Score'].mean(),   #Average of reading scores
    '% Passing Math': school_summary.groupby(['Spending Ranges (Per Student)'])['% Passing Math'].mean(),   #Average of math pass percentages
    '% Passing Reading': school_summary.groupby(['Spending Ranges (Per Student)'])['% Passing Reading'].mean(),   #Average of reading pass percentages
    '% Overall Passing Rate': school_summary.groupby(['Spending Ranges (Per Student)'])['% Overall Passing Rate'].mean()   #Average of overall pass percentages
})

    #Format results
    #Since all results are formatted in the same way, one function can be used for everything instead of applying series by series
Score_by_Spending.applymap('{:.2f}'.format)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,99.63
$585-615,83.60,83.89,94.23,95.90,99.68
$615-645,79.08,81.89,75.67,86.11,90.61
$645-675,77.00,81.03,66.16,81.13,86.40


## Scores by School Size

* Perform the same operations as above, based on school size.

In [18]:
    #Define bin ranges and labels
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [19]:
    #Create series of what bin range each each row falls under by 'School Size'
    #Append 'school_summary' DataFrame with series
school_summary['School Size'] = pd.cut(school_summary['Total Students'],size_bins,labels=group_names)

    #Create DataFrame of student score summary
    #All data is grouped by school size
Score_by_Size = pd.DataFrame({
    'Average Math Score': school_summary.groupby(['School Size'])['Average Math Score'].mean(),   #Average of math scores
    'Average Reading Score': school_summary.groupby(['School Size'])['Average Reading Score'].mean(),   #Average of reading scores
    '% Passing Math': school_summary.groupby(['School Size'])['% Passing Math'].mean(),   #Average of math pass percentages
    '% Passing Reading': school_summary.groupby(['School Size'])['% Passing Reading'].mean(),   #Average of reading pass percentages
    '% Overall Passing Rate': school_summary.groupby(['School Size'])['% Overall Passing Rate'].mean()   #Average of overall pass percentages
})

    #Format results
    #Since all results are formatted in the same way, one function can be used for everything instead of applying series by series
Score_by_Size.applymap('{:.2f}'.format)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,99.56
Medium (1000-2000),83.37,83.86,93.60,96.79,99.63
Large (2000-5000),77.75,81.34,69.96,82.77,87.94


## Scores by School Type

* Perform the same operations as above, based on school type.

In [37]:
    #Crate DataFrame of student scores
    #All data is grouped by 'School Type'
Score_by_Type = pd.DataFrame({
    'Average Math Score': school_summary.groupby(['School Type'])['Average Math Score'].mean(),   #Average of math scores
    'Average Reading Score': school_summary.groupby(['School Type'])['Average Reading Score'].mean(),   #Average of reading scores
    '% Passing Math': school_summary.groupby(['School Type'])['% Passing Math'].mean(),   #Average of math pass percentages
    '% Passing Reading': school_summary.groupby(['School Type'])['% Passing Reading'].mean(),   #Average of reading pass percentages
    '% Overall Passing Rate': school_summary.groupby(['School Type'])['% Overall Passing Rate'].mean()   #Average of overall pass percentages
})

    #Format results
    #Since all results are formatted in the same way, one function can be used for everything instead of applying series by series
Score_by_Type.applymap('{:.2f}'.format)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,99.61
District,76.96,80.97,66.55,80.80,86.27
